In [ ]:
%pip install openai pydantic python-dotenv openai instructor graphviz eval_type_backport --quiet

import os
import rich
import openai
import instructor
from pydantic import BaseModel
from typing import List
from dotenv import load_dotenv
import graphviz
from IPython.display import Image, display

load_dotenv()
DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [4]:
class Node(BaseModel):
    label: str
    entity_type: str


class Edge(BaseModel):
    source: Node
    target: Node
    label: str


class Graph(BaseModel):
    nodes: List[Node]
    edges: List[Edge]


class InferenceRequest(BaseModel):
    text: str
    graph: Graph


class InferenceResponse(BaseModel):
    response: str
    new_nodes: List[Node]
    new_edges: List[Edge]

In [ ]:

with open("assets/prompts/task-prompt.txt", "r") as file:
    task_prompt = file.read()


console = rich.console.Console()
console.rule("[bold green]Task Prompt[/bold green]")
console.print(task_prompt)
console.rule()

In [ ]:
def get_response(text):

In [ ]:
def main_loop():
    while True:
        user_input = input("You: ")
        if user_input == "/exit":
            break
        console.print(f"\n\n[bold blue]{user_input}[/bold blue] \n\n")
        print("Emma: ", user_input)

main_loop()